<table align="center">

  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/a-rebmann/nlp4bpa/blob/main/nlp4bpa_tutorial_2023.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
</table>

# NLP for BPA - Hands-on Exercises

## Outline

### 1. Event log and Process Model Analysis
    1.1 Importing and analyzing an event log with GPT4 and pm4py
    1.2 Custom task: action and object extraction from activity labels using LLMs
    1.3 Something where we have extracted semantic info already and show potential use cases. Perhaps based on extracted action and object info

### 2. Analyzing textual process descriptions
    Imperative model extraction from text
    
### 3. Future of NLP for BPA

In [2]:
# Required installs

!pip install -q pm4py==2.7.3
!pip install -q spacy
!pip install -q spacy-transformers
!pip install -q openai
!python -m spacy download en_core_web_sm
!python -m pip install spacy-llm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Importing an example event log

import pm4py
travel_event_log = pm4py.read_xes("/DomesticDeclarations.xes")